In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_Union_Councils_of_Karachi').text
soup = BeautifulSoup(source,'lxml')
uc_tab = soup.find('div', class_='toc').ul
district = []
for lis in uc_tab.find_all('li',class_='toclevel-2'):
    district.append(lis.find('span', class_='toctext').text)

In [3]:
district

['Baldia Town',
 'Bin Qasim Town',
 'Gadap Town',
 'Gulberg Town',
 'Gulshan Town',
 'Jamshed Town',
 'Kemari Town',
 'Korangi Town',
 'Landhi Town',
 'Liaquatabad Town',
 'Lyari Town',
 'Malir Town',
 'New Karachi Town',
 'North Nazimabad Town',
 'Orangi Town',
 'Saddar Town',
 'Shah Faisal Town',
 'S.I.T.E. Town (Sindh Industrial & Trading Estate)']

## for town

In [4]:
town = []
districts = [] 
dist_and_town = {}
for dis_links in district:
    source2 = requests.get('https://en.wikipedia.org/wiki/{}'.format(dis_links)).text
    soup2 = BeautifulSoup(source2,'lxml')
    try:
        neigh_tab = soup2.find('div',class_='div-col columns column-width')
        for lis2 in neigh_tab.find_all('li'):
            town.append(lis2.text)
            districts.append(dis_links)
    except Exception as e:
        pass

dist_and_town = {'Districts':districts,'Town':town}

## create dataframe

In [5]:
df_khi = pd.DataFrame(dist_and_town)
df_khi.head()

,Districts,Town
0,Bin Qasim Town,Abdullah Goth
1,Bin Qasim Town,Cattle Colony
2,Bin Qasim Town,Gaghar
3,Bin Qasim Town,Green Park City
4,Bin Qasim Town,Gulshan-e-Hadeed


In [6]:
df_khi.shape

(220, 2)

**Some Neighborhoods enter manually**

In [7]:
#save the file sp enter manually
#df_khi.to_csv('karachi.csv')

In [8]:
#read file
df_khi = pd.read_csv('karachi_district.csv')
df_khi.head()

,Districts,Town
0,Bin Qasim Town,Abdullah Goth
1,Bin Qasim Town,Cattle Colony
2,Bin Qasim Town,Gaghar
3,Bin Qasim Town,Green Park City
4,Bin Qasim Town,Gulshan-e-Hadeed


In [9]:
df_khi.shape

(294, 2)

In [16]:
import geocoder
from geopy.geocoders import Nominatim

**Find the Co-ordiantes of each Neighborhood**

In [107]:
address = 'Haroonabad ,Karachi,Pakistan'
geolocator = Nominatim(user_agent='khi_explorer')
location = geolocator.geocode(address)
la = location.latitude
lo = location.longitude
print('Latitude {} and Longitude {}'.format(la,lo))

AttributeError: 'NoneType' object has no attribute 'latitude'

In [ ]:
latitude = []
longitude = []
for t_loc in df_khi['Town']: 
    address = '{},Karachi,Pakistan'.format(t_loc)
    geolocator = Nominatim(user_agent='khi_explorer')
    location = geolocator.geocode(address,timeout=1000)
    try:
        latitude.append(location.latitude)
        longitude.append(location.longitude)
        print('Latitude {} and Longitude {}'.format(latitude,longitude))
    except Exception as e:
        latitude.append("")
        longitude.append("")

In [99]:
len(latitude)

294

In [100]:
len(longitude)

294

In [101]:
df_khi['Latitude'] = latitude
df_khi['Longitude'] = longitude
df_khi.head()

,Districts,Town,Latitude,Longitude
0,Bin Qasim Town,Abdullah Goth,24.8665,67.2837
1,Bin Qasim Town,Cattle Colony,24.8793,67.1987
2,Bin Qasim Town,Gaghar,,
3,Bin Qasim Town,Green Park City,24.8577,67.2225
4,Bin Qasim Town,Gulshan-e-Hadeed,24.87,67.3601


In [102]:
df_khi.to_csv('karachi_lat_lng.csv')

In [106]:
len(df_khi[df_khi['Latitude'] == ''])

81

In [121]:
dfr = df_khi.drop(df_khi[df_khi['Latitude'] == ""].index,axis=0)

In [123]:
len(dfr[dfr['Latitude'] == ''])

0